# Examples for MuJoCo and Bimanual Sim Usage

This notebook contains convenient introductory examples of simulation classes usage and best practices.

1. Start off by running the first cell below.
2. Then, you should be able to run any cell in the notebook individually.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.absolute()))

### Low-level MuJoCo Model Instantiation

In [23]:
import mujoco
import mujoco.viewer
from robot_descriptions import aloha_mj_description
from xml.etree import ElementTree as ET

# load the bimanual MuJoCo scene directly from the third-party package
model = mujoco.MjModel.from_xml_path(aloha_mj_description.MJCF_PATH)
data = mujoco.MjData(model)

print('data.qpos contains the positions of all joints:', data.qpos)
print('data.ctrl contains all actuator control signals:', data.ctrl)

# launch an interactive viewer
mujoco.viewer.launch(model, data)

data.qpos contains the positions of all joints: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
data.ctrl contains all actuator control signals: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### The BimanualSim Class

In [24]:
from pathlib import Path
from kinematics import parse_kinematic_chain, forward
from robot.sim import BimanualSim
from robot_descriptions import aloha_mj_description

# create the bimanual simulation, but also import a red block and noncolliding green sphere
with BimanualSim(merge_xml_files=[Path('block.xml'), Path('indicator.xml')]) as sim:
  # set the position of the green sphere
  k = parse_kinematic_chain(aloha_mj_description.MJCF_PATH, 'right/base_link', 'right/gripper_base')
  joint_pos, joint_quat = forward(k, sim.data.qpos[8:14])
  sim.data.mocap_pos[0] = joint_pos[-1]
  print('Moving the green sphere indicator to the right gripper position:', joint_pos[-1])

  # launch an interactive viewer
  sim.launch_viewer()

Moving the green sphere indicator to the right gripper position: [ 0.36478541 -0.019       0.4696409 ]


### Rollout a Handcrafted Block-Passing Policy in an Interactive Viewer

In [25]:
import mujoco
import mujoco.viewer
from pathlib import Path
from robot.sim import BimanualSim

from policy.privileged_policy import PrivilegedPolicy

with BimanualSim(merge_xml_files=[Path('block.xml')]) as sim:
  policy = PrivilegedPolicy(sim.model, sim.data)

  prev_time = sim.data.time
  obs = sim.get_obs()
  with mujoco.viewer.launch_passive(sim.model, sim.data) as viewer:
    while viewer.is_running():
      if sim.data.time < prev_time:
        sim.reset()
      prev_time = sim.data.time

      # ubiquitous obs+action sim step loop
      action = policy(obs)
      obs = sim.step(action)

      viewer.sync()

### Time a Rollout

In [26]:
import time

with BimanualSim(merge_xml_files=[Path('block.xml')]) as sim:
  policy = PrivilegedPolicy(sim.model, sim.data)

  start = time.perf_counter()
  sim.rollout(policy, 400)

print(f'One rollout of the privileged policy for 400 steps took {time.perf_counter() - start:.2f} seconds.')

One rollout of the privileged policy for 400 steps took 80.67 seconds.


### Rendering Rollout Observations

In [33]:
import os
from typing import List
import numpy as np
import cv2
from IPython.display import Video
from sim import BimanualSim

image_dims = (256, 256)  # h x w
sim_steps = 450

sim = BimanualSim(merge_xml_files=['block.xml'], camera_dims=image_dims, obs_camera_names=['wrist_cam_left', 'wrist_cam_right'])
policy = PrivilegedPolicy(sim.model, sim.data)

# rollout simulation for 450 steps
left_wrist_frames, right_wrist_frames = [], []
obs = sim.get_obs()
for sim_step in range(sim_steps):
  left_wrist_frames.append(obs.visual[0])
  right_wrist_frames.append(obs.visual[1])
  action = policy(obs)
  obs = sim.step(action)
del sim

# save frames to video
def save_frames_to_video(frames: List[np.ndarray], video_file_name: str) -> str:
  os.makedirs('out', exist_ok=True)
  video_path = f'out/{video_file_name}'
  video_writer = cv2.VideoWriter(video_path, cv2.VideoWriter.fourcc(*'H264'), 20, tuple(reversed(image_dims)))
  for frame in frames:
    video_writer.write(cv2.cvtColor(frame.astype(np.uint8), cv2.COLOR_RGB2BGR))
  video_writer.release()
  return video_path

left_wrist_video_path = save_frames_to_video(left_wrist_frames, 'example-bimanual-rollout-left.mp4')
right_wrist_video_path = save_frames_to_video(right_wrist_frames, 'example-bimanual-rollout-right.mp4')

In [ ]:
Video(left_wrist_video_path, width=image_dims[1], height=image_dims[0])

In [35]:
Video(right_wrist_video_path, width=image_dims[1], height=image_dims[0])